# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages
import re
import os
import glob
import json
import csv
import numpy as np
import pandas as pd
import cassandra

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

C:\Users\user\Documents\Udacity DE\Data modeling\data-modeling-with-apache-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
     
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        csvreader
        next(csvreader) # skip header
                
        # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# print(len(full_data_rows_list))
# print(full_data_rows_list)

# creating event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of the project. 

## Now work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code are shown in the cells below

#### Creating a Cluster

In [6]:
# Make a connection to a Cassandra instance your local machine (127.0.0.1)

from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])

# Establish connection to begin executing queries
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
query = """CREATE KEYSPACE IF NOT EXISTS sparkify_k
        WITH REPLICATION =
        {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
session.execute(query)

#### Set Keyspace

In [8]:
# Set KEYSPACE
query = """sparkify_k"""
session.set_keyspace(query)

### Create tables to run the following queries. With Apache Cassandra database tables will be modeled on the queries to be run.

## Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [9]:
## Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, \
## and itemInSession = 4

query = """CREATE TABLE IF NOT EXISTS music_session (
        session_id INT, item_in_session INT, artist TEXT, song_title TEXT, song_length FLOAT,
        PRIMARY KEY(session_id, item_in_session))"""
session.execute(query)                

In [10]:
file = 'event_datafile_new.csv'

with open(file, 'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:

        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_session (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Result
#### Table : 
* music_session

#### Columns :
* session_id 
* item_in_session 
* artist 
* song_title 
* song_length

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
rows = session.execute("""SELECT * FROM music_session WHERE session_id = 338 AND item_in_session = 4""")

for row in rows:
    print(row)

Row(session_id=338, item_in_session=4, artist='Faithless', song_length=495.30731201171875, song_title='Music Matters (Mark Knight Dub)')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) \
## for userid = 10, sessionid = 182

query = """CREATE TABLE IF NOT EXISTS event_log (
        user_id INT, session_id INT, item_in_session INT, artist TEXT, song_title TEXT,  first_name TEXT, last_name TEXT,
        PRIMARY KEY (user_id, session_id, item_in_session))"""
session.execute(query)

file = 'event_datafile_new.csv'

with open(file, 'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:                
        query = "INSERT INTO event_log (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]` 
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### Result
#### Table : 
* event_log

#### Columns :
* user_id 
* session_id  
* item_in_session 
* artist 
* song_title 
* first_name 
* last_name

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
rows = session.execute("""SELECT * FROM event_log WHERE user_id = 10 AND session_id = 182""")

for row in rows:
    print(row.item_in_session, row.artist, row.song_title, row.first_name, row.last_name)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = """CREATE TABLE IF NOT EXISTS song_users (song_title TEXT, user_id INT, first_name TEXT, last_name TEXT,
        PRIMARY KEY(song_title, user_id))"""
session.execute(query)        

file = 'event_datafile_new.csv'

# Read csv file with pandas 
df = pd.read_csv(file, usecols = [9, 10, 1, 4])
df.drop_duplicates(inplace=True)

for i, row in df.iterrows():
    
    query = "INSERT INTO song_users (song_title, user_id, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s)"

    session.execute(query, (row.song, int(row.userId), row.firstName, row.lastName))

### Result
#### Table : 
* song_users

#### Columns : 
* song_title 
* user_id
* first_name 
* last_name

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
rows = session.execute("""SELECT * FROM song_users WHERE song_title = 'All Hands Against His Own'""")

for row in rows:
    print(row)

Row(song_title='All Hands Against His Own', user_id=29, first_name='Jacqueline', last_name='Lynch')
Row(song_title='All Hands Against His Own', user_id=80, first_name='Tegan', last_name='Levine')
Row(song_title='All Hands Against His Own', user_id=95, first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [16]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS music_session") 
session.execute("DROP TABLE IF EXISTS event_log")
session.execute("DROP TABLE IF EXISTS song_users") 

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()